##### This jupyter notebooks is analyze which transcription factors or miRNAs that regulate the expression of genes using a Lasso regression model
##### Date: Feb 22, 2023

In [7]:
#import libraries
import pandas as pd
from sklearn import linear_model 
import numpy as np

In [4]:
GE = pd.read_csv("../../Data_resource/RPKM.csv",index_col = "Unnamed: 0")
GE.index = GE['Gene']

In [5]:
#gene id mapping
gene_map = GE.loc[:,['Gene','Symbol']]
dic_gene_map = {}
for i in range(0,gene_map.shape[0]):
    dic_gene_map[gene_map.iloc[i,0]] = gene_map.iloc[i,1]
    dic_gene_map[gene_map.iloc[i,1]] = gene_map.iloc[i,0]
    
# full gene list and sample list
gene_list = (list(GE.index))
sample_list = list(set(GE.columns) - set(['Gene','Symbol']))

# Formated input gene expression matrix
input_data = GE.loc[list(gene_list), sample_list]

In [6]:
def Filter_low_Expr(arr):
    '''
    Usage: This function is uded to filter out genes which has expression value logRPKM smaller than 0 in more than 50% of the samples.
    Parameter: arr, expression of one gene across different samples
    '''
    total_len = len(arr)
    count = 0
    for i in arr:
        if i > 0:
            count = count + 1
    if count > 0.5*total_len: #If 
        return(True)
    else:
        return(False)

In [8]:
# Filter genes and scale the data

scaled_df = pd.DataFrame()
array_list = list()
length = input_data.shape[0]
gene_list_remain = []
for i in range(0,length):
    gene = gene_list[i]
    arr = input_data.iloc[i,:]
    if Filter_low_Expr(arr):
        array_mean = np.mean(arr)
        array_std = np.std(arr)
        array_scale = (arr - array_mean)/array_std
        array_list.append(array_scale.values)
        gene_list_remain.append(gene)
#cur_df = pd.DataFrame({gene: array_scale})
#scaled_df = pd.concat([scaled_df, cur_df], axis =1 )
    
scaled_df = pd.DataFrame(array_list)
scaled_df.columns = sample_list
scaled_df.index = gene_list_remain

In [10]:
# Load the transcription factor and target gene pairs
TFs = pd.read_csv("~/Documents/GitHub_project/fastqpi_BigGIM/KGs/TFs/database.csv")

In [13]:
#Select TF and target genes with high or medium confidence #This might need to be revised.
TFs_sele = TFs.loc[TFs['score'].isin(['A','B','C','D','E'])]

In [14]:
miRNA_tar_KG = pd.read_csv("~/Documents/GitHub_project/fastqpi_BigGIM/KGs/miRTarBase/KG_miRNA_target_edges.csv")
miRNA_list = []
for i in set(GE['Symbol']):
    if 'MIR' in i:
        miRNA_list.append(i)
        

### select regulators for one target gene

In [243]:
result_list = []
count = 0
new_coef = []

coef_list_all = []
target_list = []
regulator_list = []

for sele_gene in gene_list_remain:
#for sele_gene in [dic_gene_map['BCL2']]:
    new_coef = []
    count = count + 1
    if count % 1000 ==0:
        print(count)
    
    #if sele_gene in set(TFs_sele['target']):
    if sele_gene in dic_gene_map:
        sele_TF_curr_gene = list(TFs_sele.loc[TFs_sele['target'] == dic_gene_map[sele_gene]]['TF'])
    
        if sele_gene in sele_TF_curr_gene:
            sele_TF_curr_gene.remove(sele_gene)
    else:
        sele_TF_curr_gene = []
        
    sele_miRNA_curr_gene = list(miRNA_tar_KG.loc[miRNA_tar_KG['Object'] == sele_gene]['Subject'])

    Potential_regulator_list = sele_TF_curr_gene + miRNA_list
    #print(len(Potential_regulator_list))
    
    X_regulators_symbol = []
    X_regulators = []
    for i in Potential_regulator_list:
        if i in dic_gene_map:
            if dic_gene_map[i] in  scaled_df.index:
                X_regulators.append(dic_gene_map[i])
                X_regulators_symbol.append(i)

    #Y_gene =dic_gene_map[sele_gene]
    Y_gene = sele_gene
    Y = scaled_df.loc[Y_gene,:].values
    X = scaled_df.loc[X_regulators,:]
    X = X.transpose()


    linreg = linear_model.Lasso(alpha=0.1) 
    linreg.fit(X,Y)
    y_pred = linreg.predict(X)

    coef = list(linreg.coef_)

    new_coef.extend(coef)
    rss = sum((y_pred-Y)**2)
    new_coef.extend([rss])
    new_coef.extend([linreg.intercept_])
    
    gene_cur = X_regulators_symbol
    gene_cur.append(sele_gene+"_"+"RSS")
    gene_cur.append(sele_gene+"_"+"intercept_") 
    
    result_cur = pd.DataFrame({"Regulator":X_regulators_symbol,
                                "coef":new_coef,
                               "target_gene":[sele_gene]*len(new_coef)})
    result_cur = result_cur.loc[result_cur['coef']!=0]
    regulator_list.extend(result_cur['Regulator'].values)
    target_list.extend(result_cur['target_gene'].values)
    coef_list_all.extend( result_cur['coef'].values)

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000


In [245]:
result_df = pd.DataFrame({"Target_gene":target_list, "Regulator":regulator_list, "Coef": coef_list_all})

In [246]:
result_df.to_csv("KG_regulatory_graph.csv")

In [248]:
Symbol_list = []
for gene in list(result_df['Target_gene']):
    if gene in dic_gene_map:
        Symbol_list.append(dic_gene_map[gene])
    else:
        Symbol_list.append(gene)
        
result_df['Target_symbol'] = Symbol_list
result_df.to_csv("KG_regulatory_graph.csv")

# End